# 실전 문제 해결 (부족한 데이터셋)

In [2]:
import tensorflow as tf
from imblearn.over_sampling import BorderlineSMOTE
import numpy as np
import random

## 하이퍼 파라미터

In [3]:
EPOCHS = 100

## 모델 정의

In [4]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## 데이터셋 준비 (불균형한 데이터셋)

In [5]:
cifar10 = tf.keras.datasets.cifar10 # 32x32x3

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# TODO: 학습 데이터를 Imbalanced small dataset으로 변형하기
x_train_small = list()
y_train_small = list()
for x, y in zip(x_train, y_train):
    if (y == 0 and random.randint(0, 100) < 10) or y == 1:
        x_train_small.append(x[:])
        y_train_small.append(y)

x_test_small = list()
y_test_small = list()
for x, y in zip(x_test, y_test):
    if y == 0 or y == 1:
        x_test_small.append(x[:])
        y_test_small.append(y)

x_train = np.stack(x_train_small, axis=0)
y_train = np.stack(y_train_small, axis=0)

x_test = np.stack(x_test_small, axis=0)
y_test = np.stack(y_test_small, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

170500096/170498071 [==============================] - 32s 0us/step


## Keras API 모델 학습 (불균형한 데이터셋)

In [6]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Train for 173 steps, validate for 63 steps
Epoch 1/100
173/173 [==============================] - 3s 20ms/step - loss: 0.4457 - accuracy: 0.8972 - precision: 0.9136 - recall: 0.9794 - val_loss: 0.6234 - val_accuracy: 0.6835 - val_precision: 0.6174 - val_recall: 0.9650
Epoch 2/100
173/173 [==============================] - 2s 10ms/step - loss: 0.2529 - accuracy: 0.9135 - precision: 0.9249 - recall: 0.9848 - val_loss: 0.6010 - val_accuracy: 0.7100 - val_precision: 0.6382 - val_recall: 0.9700
Epoch 3/100
173/173 [==============================] - 2s 11ms/step - loss: 0.2427 - accuracy: 0.9146 - precision: 0.9301 - recall: 0.9796 - val_loss: 0.7173 - val_accuracy: 0.6325 - val_precision: 0.5774 - val_recall: 0.9880
Epoch 4/100
173/173 [==============================] - 2s 10ms/step - loss: 0.2281 - accuracy: 0.9226 - precision: 0.9328 - recall: 0.9858 - val_loss: 0.4985 - val_accuracy: 0.7805 - val_precision: 0.7082 - val_recall: 0.9540
Epoch 5/100
173/173 [==============================] 

## 데이터셋 준비 (BorderlineSMOTE)

In [7]:
# TODO: BorderlineSMOTE 적용하기
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1] * x_train.shape[2] * x_train.shape[3])).astype(np.float32)
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1] * x_test.shape[2] * x_test.shape[3])).astype(np.float32)

smote = BorderlineSMOTE()
x_train, y_train = smote.fit_resample(x_train, y_train)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## Keras API 모델 학습 (BorderlineSMOTE)

In [8]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Train for 313 steps, validate for 63 steps
Epoch 1/100
313/313 [==============================] - 7s 24ms/step - loss: 0.6312 - accuracy: 0.7145 - precision: 0.7282 - recall: 0.6844 - val_loss: 0.4480 - val_accuracy: 0.8070 - val_precision: 0.7693 - val_recall: 0.8770
Epoch 2/100
313/313 [==============================] - 7s 21ms/step - loss: 0.4460 - accuracy: 0.7966 - precision: 0.8200 - recall: 0.7600 - val_loss: 0.3924 - val_accuracy: 0.8205 - val_precision: 0.8487 - val_recall: 0.7800
Epoch 3/100
313/313 [==============================] - 7s 21ms/step - loss: 0.3695 - accuracy: 0.8446 - precision: 0.8755 - recall: 0.8034 - val_loss: 0.3882 - val_accuracy: 0.8270 - val_precision: 0.8330 - val_recall: 0.8180
Epoch 4/100
313/313 [==============================] - 7s 21ms/step - loss: 0.3260 - accuracy: 0.8675 - precision: 0.8992 - recall: 0.8278 - val_loss: 0.3820 - val_accuracy: 0.8290 - val_precision: 0.8647 - val_recall: 0.7800
Epoch 5/100
313/313 [==============================] 